In [1]:
import streamlit as st
import pickle
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, root_mean_squared_error

# title and user input
st.title("Apple Stock Price Prediction Using Triple Exponential smoothing")

st.subheader("Input")
forecast_days = st.number_input("Number of days to forecast", min_value=1,max_value=365,value=30)


#load data (using st.cache_data for caching)
@st.cache_data
def load_data():
    data= pd.read_csv("dataAAPL.csv",index_col="Date",parse_dates=["Date"])
    return data
data=load_data()


## historical data display
st.write("Historical stock prices")
st.line_chart(data['Close'])

# Model training and forecasting
if st.button("Model training and forecasting"):
    try:
        # 1. Train the model on the *differenced*
        model= ExponentialSmoothing(data['Close_diff'].dropna(), trend='additive', seasonal='additive', seasonal_periods=12)
        results = model.fit()

        with open('triple_exp.pkl','wb') as file:
            pickle.dump(model, file)
            
        forecast = results.forecast(steps=forecast_days) 

        # 2. Inverse transform the forecast (convert back to original scale)
        forecast_original_scales=forecast.cumsum() + data['Close'].iloc[-1]


        # 3. Create forecast DataFrame with correct dates
        forecast_dates=pd.date_range(start=data.index[-1] + pd.Timedelta(days=1),periods=forecast_days)
        
        forecast_TES=pd.DataFrame({
            'Date': forecast_dates,
            'Forecast': forecast_original_scales
        }).set_index('Date') 

        # 4. Display the forecast
        st.subheader("Forecasted Stock Prices")
        st.write(forecast_TES)

        # 5. Plot historical data and forecast
        fig, ax = plt.subplots(figsize=(10, 6))
        data['Close'].plot(ax=ax, label='Historical Data')
        forecast_TES['Forecast'].plot(ax=ax, label='Forecast', color='red')
        ax.set_xlabel("Date")
        ax.set_ylabel("Stock Price")
        ax.legend()
        st.pyplot(fig)
        # 6. Calculate metrics
        mae = mean_absolute_error(data['Close'].iloc[-forecast_days:], forecast_original_scales)  # Compare with actual values
        mse = mean_squared_error(data['Close'].iloc[-forecast_days:], forecast_original_scales)
        rmse = np.sqrt(mse)

        # 7
        . Create a DataFrame for the metrics
        metrics_df = pd.DataFrame({
            'Metric': ['MAE', 'MSE','RMSE'],
            'Value': [mae, mse, rmse]
        })

        # Display the metrics DataFrame
        st.subheader("Model Evaluation Metrics")
        st.dataframe(metrics_df)  # Or st.table(metrics_df) for a static table

    except Exception as e:  # Catch any exceptions during training/forecasting
        st.error(f"An error occurred: {e}")  # Display error message
        st.stop()







2025-01-30 11:38:00.182 
  command:

    streamlit run C:\Users\shrey\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-01-30 11:38:00.182 No runtime found, using MemoryCacheStorageManager
2025-01-30 11:38:00.198 No runtime found, using MemoryCacheStorageManager
